In [1]:
from pathlib import Path
secrets_path = Path('/Users/calm/home/notion_assistant/scratch/telegram_bot_example/secrets.json')
import json
secrets = json.load(secrets_path.open())

In [2]:
import requests, json
import datetime

In [3]:
# actionable_db_url
# notes_db_url = 

# prod: DB_Actionable
# todos_database = 'https://www.notion.so/lavrovs/2290411c436848479b4826e63d740c63?v=f50c1eebf2ba4441a922d523587b6b66'

# test: DB_test_notion_assistant
todos_database_url = 'https://www.notion.so/lavrovs/d96416324b44433a9d378f0767627301?v=877e567e513a4583b9e4614d1b059ba1'


# --------------------------------------
# utils
# --------------------------------------

def extract_database_id(database_link):
    """
    :param database_link: url to the database page
    :type database_link: str
    :return: Id of the database for use in Notion API
    """
    return database_link.rsplit('/', 1)[-1].split('?', 1)[0]

# def get_db_id(db_url):
#     return 

In [39]:

    
def get_database(database_id, headers):
    url = f"https://api.notion.com/v1/databases/{database_id}"
    res = requests.request("GET", url, headers = headers).json()
    return res
    

def get_page(page_id, headers):
    url = f"https://api.notion.com/v1/pages/{page_id}"
    res = requests.request("GET", url, headers = headers).json()
    return res
    
def get_database_pages(database_id, headers):
    url = f"https://api.notion.com/v1/databases/{database_id}/query"
    res = requests.request("POST", url, headers = headers).json()
    
    data = res
    while res['has_more']:
        res = requests.request("POST", metadata_request_url, next_cursor=res['next_cursor'], headers = self.workspace.headers).json()
        data['results'] += res['results']
        
    return data

def create_page(data, headers):
    CREATE_URL = 'https://api.notion.com/v1/pages'
    
    data = json.dumps(data)
    res = requests.request("POST", CREATE_URL, headers=headers, data=data)
    return res

class NotionWorkspace:
    def __init__(self, token):
        self.token = token
    
    # 
    def show_map(self, depth=2):
        raise NotImplemented("Display structure of a page")

    @property
    def headers(self):
         return {
            "Authorization": "Bearer " + self.token,
            "Content-Type": "application/json",
            "Notion-Version": "2022-02-22" # todo: figure out notion version.
        }
    #     def __getattr__(self, key):
    #         # todo: support database dot access
    #         pass

    #     def __dir__(self):
    #         # todo: show all available databases
    #         pass

    # ------------------
    # getting a database
    # ------------------
    
    # core: get database by id.
    
    # todo: find database by name
    def get_page(self, page_id):
        return get_page(page_id, self.headers)

class NotionDB:
    def __init__(self, database_id: str, workspace: NotionWorkspace):
        self.database_id = database_id
        self.workspace = workspace
        
        #get metadata
        self.metadata = get_database(self.database_id, self.workspace.headers)
        
    @property
    def name(self):
        raise NotImplemented("Get a name of a database")
    
    def load_all_pages(self):
        self._load_pages()
    
    def _load_pages(self, limit=None):
        # todo: support pagination for more pages.
        self.data = get_database_pages(self.database_id, self.workspace.headers)
#         print(res.status_code)
        # print(res.text)

#         with open('./db.json', 'w', encoding='utf8') as f:
#             json.dump(data, f, ensure_ascii=False)
    
    @property
    def schema(self):
        return self.metadata['properties']
    
    def _verify_properties(self, properties):
        # verify keys
        for key in properties:
            assert key in self.schema
        # todo: verify values
        
    def format_value(self, key, value):
        if self.schema[key]['type'] in {'title', 'rich_text'}:
            return [{"text": { "content": value}}]
        else:
            raise NotImplemented("")
    
    def create_page(self, **properties):
        # todo: verify that properties 
        self._verify_properties(properties)

        # format properties
        request_properties = {}
        for key, value in properties.items():
            request_properties[key] = {
                self.schema[key]['type']: self.format_value(key, value)
            }
            
        new_page_data = {
            "parent": { "database_id": self.database_id },
            "properties": request_properties
        }

        res = create_page(new_page_data, self.workspace.headers)

        print(res.status_code)
        print(res.text)
        
    def update_page(self):
        updateUrl = f"https://api.notion.com/v1/pages/{pageId}"

        updateData = {
            "properties": {
                "Value": {
                    "rich_text": [
                        {
                            "text": {
                                "content": "Pretty Good"
                            }
                        }
                    ]
                }        
            }
        }

        data = json.dumps(updateData)

        response = requests.request("PATCH", updateUrl, headers=headers, data=data)

        print(response.status_code)
        print(response.text)
        
    def get_properties(self):
        raise NotImplemented("Return dict with db structure")
        
    def show_properties(self):
        raise NotImplemented("Todo: display structure of a database in a comprehensive way")
        
        
class NotionPage:
    def __init__(self, page_id: str, database: NotionDB or str = None, workspace=None):
        self.page_id = page_id
        self.database = database
    
    def __dir__(self):
        # todo: autocomplete page properties names from database structure
        pass


In [40]:
notion_token = secrets['notion_token']

ws = NotionWorkspace(token=notion_token)

db_id = extract_database_id(todos_database_url)
db = NotionDB(database_id=db_id, workspace=ws)

In [46]:
# db.create_page(Name="test", bah='test2')

In [45]:
# db.load_all_pages()

In [44]:
# db.data.keys()

In [43]:
# db.data['results'][0]

In [42]:
# db.data